<div style="text-align: center;">
    <h1> </font> <font color = #4854E8>Data Preprocessing</h2> </font>
</div>

> The data preprocessing stage included cleaning the data to remove any inconsistencies or errors, dealing with missing data using imputation, and handling outliers that might skew the results through the "capping" method. Furthermore, temporal and seasonal parameters were developed to capture the time-dependent patterns in the data, and weather-related features were retrieved to incorporate climatic impacts on energy demand. Moreover, trends and seasonality were removed from the target variable in order to effectively isolate the underlying patterns. Categorical variables were encoded to incorporate them in machine learning models, and the data were resampled at an hourly frequency for consistency and ease of analysis. To carry out these preparation steps, several Python tools were used, including Pandas for data manipulation, NumPy for numerical operations, and Scikit-Learn for categorical variable encoding. Additionally, the Statsmodels library was used for seasonal decomposition analysis to identify seasonal components in the data.

**The data obtained from the Bureau of Meteorology (BOM) automatic weather stations in Australia, which provide half-hourly temperature measurements spanning two decades in the capital cities of Hobart, Melbourne, Adelaide, Sydney, and Brisbane. And half-hourly energy demand measurements across five states: South Australia, Victoria, New South Wales, Queensland, and Tasmania.**

---

# 01- Importing required libraries

In [1]:
import os
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None

from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.seasonal import seasonal_decompose

import warnings
warnings.filterwarnings("ignore")

# 02- Importing datasets

There are many temperature and energy spreadsheets formatted as CSVs. Although there's an incredible number of them, it is just because the data was divided into small chunks. Each CSV is a continuation of the last one. The temperature spreadsheets contain dates, along with a variety of temperature, humidity and precipitation measurements.The energy files contain dates, energy demand history, prices (RRP) and whether the data was manually or automatically logged. The measurements have been made on a 30-minute basis.

Following code reads those multiple CSV files from specified directories, creates Pandas DataFrames for each CSV file, and then concatenates these DataFrames into two main DataFrames (energy and temperature). These DataFrames contain the combined data from all the CSV files, for further analysis and processing.

In [2]:
# Define the paths to the directories containing the CSV files
data_directory = "C:/Users/HP/Documents/NIBM/Year 3/Research Project/1- My Project Report/Model Training and Prediction/Data"

# Retrieve a list of filenames present in the energy and temperature directories
energy_directory = os.path.join(data_directory, "Energy")
temperature_directory = os.path.join(data_directory, "Temperature")

energy_locations = os.listdir(energy_directory)
temperature_locations = os.listdir(temperature_directory)

# Read each CSV file present in the Energy directory and create a list of DataFrames
energy_CSVs = [pd.read_csv(os.path.join(energy_directory, location)) for location in energy_locations]

# Read each CSV file present in the Temperature directory (only files containing "Data" in their filename) and create a list of DataFrames
temperature_CSVs = [pd.read_csv(os.path.join(temperature_directory, location)) for location in temperature_locations if "Data" in location]

In [3]:
energy = pd.concat(energy_CSVs, ignore_index=True)
temperature = pd.concat(temperature_CSVs, ignore_index=True)

In [4]:
energy.columns
temperature.columns

Index(['REGION', 'SETTLEMENTDATE', 'TOTALDEMAND', 'RRP', 'PERIODTYPE'], dtype='object')

Index(['hm', 'Station Number', 'Year Month Day Hour Minutes in YYYY', 'MM',
       'DD', 'HH24', 'MI format in Local time',
       'Year Month Day Hour Minutes in YYYY.1', 'MM.1', 'DD.1', 'HH24.1',
       'MI format in Local standard time',
       'Precipitation since 9am local time in mm',
       'Quality of precipitation since 9am local time',
       'Air Temperature in degrees C', 'Quality of air temperature',
       'Wet bulb temperature in degrees C', 'Quality of Wet bulb temperature',
       'Dew point temperature in degrees C',
       'Quality of dew point temperature', 'Relative humidity in percentage %',
       'Quality of relative humidity', 'Wind speed in km/h',
       'Wind speed quality', 'Wind direction in degrees true',
       'Wind direction quality',
       'Speed of maximum windgust in last 10 minutes in  km/h',
       'Quality of speed of maximum windgust in last 10 minutes',
       'Mean sea level pressure in hPa', 'Quality of mean sea level pressure',
       'Stati

# 03- Preprocessing the energy and temperature data by cleaning, transforming, and structuring 

The following data cleaning steps defines two functions, preprocess_energy and preprocess_temperature, which perform data preprocessing tasks on Pandas DataFrames representing energy and temperature data respectively.

- For energy data, the function renames columns, removes columns with mostly constant values, drops duplicate rows, maps region codes to their corresponding names, converts date strings to datetime objects, sets the datetime column as the index, and removes the 'RRP' column.

- For temperature data, the function renames columns, removes columns with mostly constant values, drops duplicate rows, drops unnecessary columns, maps station numbers to region names, converts date components to a datetime object, replaces "###" values in the 'RelativeHumidity%' column with NaN, strips leading and trailing whitespaces from numeric columns, converts numeric columns to numeric data types, and sets the datetime column as the index. Finally, the relevant time-related columns such as 'Year', 'Month', 'Day', 'Hour', and 'Minute' are converted to integer data types to conserve memory.

In [5]:
def preprocess_energy(energy):
    
    # Identify columns with two or fewer unique values (mostly constant) and remove them
    remove_energy = [name for name, series in energy.items() if len(series.unique()) <= 2]
    energy.drop(remove_energy, axis=1, inplace=True)
    
    # Drop duplicate rows
    energy.drop_duplicates(inplace=True)
    
    # Drop unnecessary columns
    energy.drop(["RRP"], axis=1, inplace=True)
    
    # Rename columns
    energy.columns = ["Region", "Date", "TotalDemand"]
    
    # Rename Region
    region_remove_number_map = {"SA1": "SA", "QLD1": "QLD", "NSW1": "NSW", "VIC1": "VIC", "TAS1": "TAS"}
    energy["Region"] = energy["Region"].map(region_remove_number_map)
    
    # Convert Region column to categorical data type
    energy['Region'] = pd.Categorical(energy['Region'])
    
    # Convert 'Date' column to the aatetime format
    energy["Date"] = pd.to_datetime(energy["Date"])
    
    # Set 'Date' column as index
    energy.set_index("Date", inplace=True)
    
    
    return energy

In [6]:
energy_data = preprocess_energy(energy)
energy_data

,Region,TotalDemand
Date,,
2000-01-01 00:30:00,NSW,6763.57000
2000-01-01 01:00:00,NSW,6386.10167
2000-01-01 01:30:00,NSW,5990.79500
2000-01-01 02:00:00,NSW,5655.97667
2000-01-01 02:30:00,NSW,5283.83667
...,...,...
2019-12-31 22:00:00,VIC,4129.96000
2019-12-31 22:30:00,VIC,4083.66000
2019-12-31 23:00:00,VIC,4104.95000


In [7]:
def preprocess_temperature(temperature):
    
    # Rename columns
    temperature.columns = [
        "HM", "StationNumber", "Year1", "Month1", "Day1", "Hour1", "Minute1", "Year", "Month", "Day", "Hour", "Minute", "Precipitation", "PrecipitationQuality",
        "AirTemperature", "AirTemperatureQuality", "WetBulbTemperature", "WetBulbTemperatureQuality", "DewTemperature", "DewTemperatureQuality", "RelativeHumidity%",
        "RelativeHumidityQuality", "WindSpeed", "WindSpeedQuality", "WindDirection", "WindDirectionQuality", "WindgustSpeed", "WindgustSpeedQuality", "SeaPressure",
        "SeaPressureQuality", "StationPressure", "StationPressureQuality", "AWSFlag", "#"
    ]
    
    # Identify columns with two or fewer unique values (mostly constant) and remove them
    remove_temperature = [name for name, series in temperature.items() if len(series.unique()) <= 2]
    temperature.drop(remove_temperature, axis=1, inplace=True)

    # Drop duplicate rows
    temperature.drop_duplicates(inplace=True)

    # Drop unnecessary columns
    temperature.drop(["Year1", "Month1", "Day1", "Hour1", "Minute1"], axis=1, inplace=True)
    temperature.drop(["AWSFlag"], axis=1, inplace=True)
    
    # Convert to datetime and create a 'Date' column
    temperature["Date"] = pd.to_datetime(temperature[["Year", "Month", "Day", "Hour", "Minute"]])

    # Map StationNumber to Region
    station_to_region_map = {23090: "SA", 40913: "QLD", 66062: "NSW", 86071: "VIC", 94029: "TAS", 86338: "VIC"}
    temperature["Region"] = temperature["StationNumber"].map(station_to_region_map)
    temperature.drop("StationNumber", axis=1, inplace=True)

    # Replace "###" in RelativeHumidity with NaN
    temperature["RelativeHumidity%"] = temperature["RelativeHumidity%"].replace("###", np.NaN)

    # Dealing with leading and trailing white spaces in numeric columns
    numeric_columns = ['Precipitation', 'AirTemperature', 'WetBulbTemperature', 'DewTemperature',
                        'RelativeHumidity%', 'WindSpeed', 'WindDirection', 'WindgustSpeed',
                        'SeaPressure', 'StationPressure']
    for column in numeric_columns:
        temperature[column] = temperature[column].str.strip()
        temperature[column] = pd.to_numeric(temperature[column], errors='coerce')
        
    # Set 'Date' column as index
    temperature.set_index("Date", inplace=True)
    
    # Convert Region column to categorical data type
    temperature['Region'] = pd.Categorical(temperature['Region'])
    
    return temperature

In [8]:
temperature_data = preprocess_temperature(temperature)
temperature_data

,Year,Month,Day,Hour,Minute,Precipitation,AirTemperature,WetBulbTemperature,DewTemperature,RelativeHumidity%,WindSpeed,WindDirection,WindgustSpeed,SeaPressure,StationPressure,Region
Date,,,,,,,,,,,,,,,,
2000-01-01 00:00:00,2000,1,1,0,0,NaN,13.1,10.0,6.6,65.0,NaN,190.0,24.1,1018.9,1012.8,SA
2000-01-01 00:30:00,2000,1,1,0,30,NaN,13.2,10.1,6.8,65.0,NaN,200.0,16.6,1018.7,1012.6,SA
2000-01-01 01:00:00,2000,1,1,1,0,NaN,13.4,10.1,6.5,63.0,NaN,180.0,20.5,1018.5,1012.4,SA
2000-01-01 01:30:00,2000,1,1,1,30,NaN,13.2,10.0,6.5,64.0,NaN,170.0,18.4,1018.3,1012.2,SA
2000-01-01 02:00:00,2000,1,1,2,0,NaN,13.0,9.7,6.0,62.0,NaN,150.0,20.5,1018.3,1012.2,SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-20 07:00:00,2020,1,20,7,0,0.0,NaN,12.5,NaN,NaN,NaN,130.0,20.5,NaN,1005.3,TAS
2020-01-20 07:30:00,2020,1,20,7,30,0.0,NaN,12.4,NaN,NaN,NaN,140.0,18.4,NaN,1005.4,TAS
2020-01-20 08:00:00,2020,1,20,8,0,0.0,NaN,12.4,NaN,NaN,NaN,150.0,24.1,NaN,1005.6,TAS


##  Calculating upper and lower bounds before the na values imputation

The following code groups weather data by region and computes the lower and upper bounds for outlier detection using the Interquartile Range (IQR) method for five different weather-related variables: AirTemperature, WetBulbTemperature, DewTemperature, SeaPressure, and StationPressure. It defines a function to calculate these bounds based on the IQR formula, then applies this function to each column separately for each region. The resulting bounds are stored in separate variables, providing a clear understanding of the data distribution and potential outliers before any data imputation or modification occurs. These bounds serve as valuable insights for quality control and anomaly detection in the dataset.

In [9]:
# Group the data by "Region"
grouped_data = temperature_data.groupby('Region')

# Define a function to calculate lower and upper bounds for a specific column
def calculate_bounds(group, column_name):
    Q1 = group[column_name].quantile(0.25)
    Q3 = group[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

# Apply the function to each column separately and store the results in separate variables
AirTemperature_bounds_before_imputation = grouped_data.apply(calculate_bounds, column_name='AirTemperature')
WetBulbTemperature_bounds_before_imputation = grouped_data.apply(calculate_bounds, column_name='WetBulbTemperature')
DewTemperature_bounds_before_imputation = grouped_data.apply(calculate_bounds, column_name='DewTemperature')
SeaPressure_bounds_before_imputation = grouped_data.apply(calculate_bounds, column_name='SeaPressure')
StationPressure_bounds_before_imputation = grouped_data.apply(calculate_bounds, column_name='StationPressure')

# 04- Dealing with missing data

The following function is designed to address missing data within the temperature_data DataFrame, particularly focusing on interpolation and replacement strategies. It first identifies columns with missing values by calculating the percentage of null values for each column. These columns are then selected for interpolation using the "time" method, which fills missing values by linearly interpolating based on time. Additionally, missing values in the 'Precipitation' column are filled with 0, presumably indicating no precipitation when data is missing. Furthermore, certain columns ('WindSpeed', 'WindDirection', and 'WindgustSpeed') are deemed unnecessary and are consequently removed from the DataFrame.

In [10]:
def impute_missing_values(temperature_data):
    
    # Calculate the percentage of null values for each column
    missing_columns = temperature_data.isnull().mean()[temperature_data.isnull().mean() > 0].keys()

    # Interpolate missing values in the identified columns using the "time" method
    temperature_data[missing_columns] = temperature_data[missing_columns].interpolate(method="time")

    # Replace remaining null values in the 'Precipitation' column with 0
    temperature_data['Precipitation'].fillna(0, inplace=True)
    
    remove_columns = ["WindSpeed", "WindDirection", "WindgustSpeed"]
    temperature_data.drop(remove_columns, axis=1, inplace=True)

    return temperature_data

In [11]:
temperature_data = impute_missing_values(temperature_data)

In [12]:
# Check for missing values in each column
missing_values = temperature_data.isnull().sum()

# Display the columns with missing values
print("Columns with missing values:")
print(missing_values[missing_values > 0])

Columns with missing values:
Series([], dtype: int64)


# 05- Dealing with outliers

When dealing with outliers, I utilized Outlier capping method, also known as winsorization or trimming, is a method used to handle extreme values (outliers) in a dataset. Outliers are data points that deviate significantly from the rest of the data, potentially skewing statistical analyses or machine learning models. Capping involves setting a threshold beyond which values are adjusted to be equal to the threshold.

The following function cap_outliers defines that for a specific region and column based on above pre-defined bounds. It then applies this function to each row in the DataFrame temperature_data for various temperature-related columns like 'AirTemperature', 'WetBulbTemperature', 'DewTemperature', and pressure-related columns like 'SeaPressure', 'StationPressure'. The outlier capping is performed separately for each column, considering the region specified in the DataFrame. For each region, it compares the imputed values to the bounds calculated before imputation. If an imputed value falls outside the corresponding bounds, replace it with the nearest bound. After capping, the original columns are dropped, and the capped columns are renamed to replace the original ones. 

In [13]:
# Function to cap outliers for a specific region and column using bounds before imputation
def cap_outliers(region, value, bounds):
    lower_bound, upper_bound = bounds[region]
    if value < lower_bound:
        return lower_bound
    elif value > upper_bound:
        return upper_bound
    else:
        return value

# Apply capping to each row in the DataFrame for each column separately using bounds before imputation
temperature_data['Capped_AirTemperature'] = temperature_data.apply(lambda row: cap_outliers(row['Region'], row['AirTemperature'], AirTemperature_bounds_before_imputation), axis=1)
temperature_data['Capped_WetBulbTemperature'] = temperature_data.apply(lambda row: cap_outliers(row['Region'], row['WetBulbTemperature'], WetBulbTemperature_bounds_before_imputation), axis=1)
temperature_data['Capped_DewTemperature'] = temperature_data.apply(lambda row: cap_outliers(row['Region'], row['DewTemperature'], DewTemperature_bounds_before_imputation), axis=1)
temperature_data['Capped_SeaPressure'] = temperature_data.apply(lambda row: cap_outliers(row['Region'], row['SeaPressure'], SeaPressure_bounds_before_imputation), axis=1)
temperature_data['Capped_StationPressure'] = temperature_data.apply(lambda row: cap_outliers(row['Region'], row['StationPressure'], StationPressure_bounds_before_imputation), axis=1)

# Drop the original columns
temperature_data.drop(columns=['AirTemperature', 'WetBulbTemperature', 'DewTemperature', 'SeaPressure', 'StationPressure'], inplace=True)

# Rename the capped columns
temperature_data.rename(columns={'Capped_AirTemperature': 'AirTemperature', 
                                 'Capped_WetBulbTemperature': 'WetBulbTemperature', 
                                 'Capped_DewTemperature': 'DewTemperature', 
                                 'Capped_SeaPressure': 'SeaPressure', 
                                 'Capped_StationPressure': 'StationPressure'}, inplace=True)

# 06- Generating Temporal and Seasonal related Parameters
The following function improves the temperature_data DataFrame by deriving several time-related features from its datetime index. Initially, it captures the day of the week, week of the month, quarter of the year, and day of the year, providing granularity in understanding temperature variations over time. Additionally, it maps each month to its corresponding season, facilitating the exploration of seasonal trends. Furthermore, it categorizes observations into distinct time periods such as night, morning, afternoon, and evening, offering insights into diurnal temperature patterns. The function also generates binary indicators for weekends, seasons (summer and winter), daytime, and holidays, enabling the identification of temporal patterns and anomalies in temperature data. 

In [14]:
def get_time_related_features(temperature_data):
    
    # Extract the day of the week as names
    temperature_data['DayOfWeek'] = temperature_data.index.strftime('%A')
    
    # Calculate the week of the month 
    temperature_data['WeekOfMonth'] = (temperature_data.index.day - 1) // 7 + 1
    
    # Extract the quarter of the year 
    temperature_data['Quarter'] = temperature_data.index.quarter
    
    # Extract the day of the month
    temperature_data['DayOfYear'] = temperature_data.index.dayofyear
    
    # Map each month to a corresponding season
    temperature_data['Season'] = temperature_data.index.month.map({
        1: 'Summer', 2: 'Summer', 3: 'Autumn',
        4: 'Autumn', 5: 'Autumn', 6: 'Winter',
        7: 'Winter', 8: 'Winter', 9: 'Spring',
        10: 'Spring', 11: 'Spring', 12: 'Summer'
    })
    
    # Apply the lambda function to create the TimeOfDay column 
    temperature_data['TimeOfDay'] = temperature_data['Hour'].apply(lambda hour: 
                                                              'Morning' if 5 <= hour < 12 
                                                              else ('Afternoon' if 12 <= hour < 17 
                                                                    else ('Evening' if 17 <= hour < 21 
                                                                          else 'Night')))

    # Create binary feature for weekends and weekdays
    temperature_data['IsWeekend'] = (temperature_data['DayOfWeek'] == 'Saturday') | (temperature_data['DayOfWeek'] == 'Sunday')
    
    return temperature_data

In [15]:
temperature_data = get_time_related_features(temperature_data)

# 07- Generating weather related features

The following function enhances the temperature_data DataFrame by deriving a range of weather-related features, providing valuable insights into atmospheric conditions. First, it calculates the difference between air temperature and dew temperature, offering an indication of atmospheric moisture levels. Then, it computes the difference between sea pressure and station pressure, reflecting variations in atmospheric pressure. Additionally, it assesses the difference in air temperature between daytime and nighttime, capturing diurnal temperature fluctuations. Moreover, it categorizes air temperature, precipitation intensity, station pressure, and relative humidity into discrete levels, enabling to capture the potential non-linear effects between weather features and energy consumption.

In [16]:
def get_weather_related_features(temperature_data):
    
    # The difference between AirTemperature and DewTemperature
    temperature_data['TemperatureDifference'] = temperature_data['AirTemperature'] - temperature_data['DewTemperature']
    
    # The difference between sea pressure and station pressure.
    temperature_data['PressureDifference'] = temperature_data['SeaPressure'] - temperature_data['StationPressure']
    
    # Create a feature that represents the difference in air temperature between day and night.
    day_avg_temp = temperature_data.loc[temperature_data['TimeOfDay'].isin(['Morning', 'Afternoon']), 'AirTemperature'].mean()
    night_avg_temp = temperature_data.loc[temperature_data['TimeOfDay'].isin(['Evening', 'Night']), 'AirTemperature'].mean()
    DayNightTempDifference = day_avg_temp - night_avg_temp
    temperature_data['DayNightTempDifference'] = DayNightTempDifference
    
    # categorical variable indicating the temperature levels
    bins1=[-np.inf, 0, 10, 20, 30, np.inf]
    temperature_levels = ['Very Cold', 'Cold', 'Mild', 'Warm', 'Hot']
    temperature_data['TemperatureLevel'] = pd.cut(temperature_data['AirTemperature'], bins=bins1, labels=temperature_levels)
    
    # categorical variable indicating the intensity of precipitation
    bins2=[-np.inf, 0, 8, 20, np.inf]
    precipitation_levels = ['No Precipitation', 'Light Precipitation', 'Moderate Precipitation', 'Heavy Precipitation']
    temperature_data['PrecipitationLevel'] = pd.cut(temperature_data['Precipitation'], bins=bins2, labels=precipitation_levels)
    
    # categorical variable indicating the pressure levels
    bins3=[-np.inf, 1000, 1030, np.inf]
    pressure_levels = ['Low Pressure', 'Normal Pressure', 'High Pressure']
    temperature_data['PressureLevel'] = pd.cut(temperature_data['StationPressure'], bins=bins3, labels=pressure_levels)
    
    # create bins for relative humidity to categorize it into different levels, indicating the amount of moisture in the air
    bins4 = [0, 30, 70, np.inf]
    humidity_levels = ['Low Moist', 'Moderate Moist', 'High Moist']
    temperature_data['HumidityLevel'] = pd.cut(temperature_data['RelativeHumidity%'], bins=bins4, labels=humidity_levels)
    
    # Convert float columns to float32
    float_columns = ['TemperatureDifference', 'PressureDifference', 'DayNightTempDifference']
    temperature_data[float_columns] = temperature_data[float_columns].astype(np.float32)
    
    return temperature_data

In [17]:
temperature_data= get_weather_related_features(temperature_data)

# 08- Combining energy and temperature data

Initially, the function sorts both the energy and temperature dataframes by their indices to ensure consistency. Subsequently, the pd.merge_asof function is employed to merge the two dataframes based on the shared column "Region," effectively combining relevant information from both datasets. A tolerance of 30 minutes is specified to allow for small temporal differences between entries from the two datasets. To handle missing values in the merged dataframe, the interpolate method with the 'pad' option is applied, ensuring that missing values are filled based on the nearest non-missing values (forward fill). Finally, certain columns such as Year, Month, Day, and Hour are converted to appropriate integer data types (e.g., np.int16 and np.uint8) to optimize memory usage and ensure data consistency.

In [18]:
def create_master_table(energy_data, temperature_data):
    
    # Sort dataframes by index
    energy_data.sort_index(inplace=True)
    temperature_data.sort_index(inplace=True)

    # Merge dataframes using pd.merge_asof
    master_table = pd.merge_asof(
        energy_data, temperature_data, left_index=True, right_index=True, by="Region", tolerance=pd.Timedelta("30 min")
    )

    # Interpolate missing values using the nearest non-missing values (forward fill)
    master_table = master_table.interpolate(method='pad')
    
    # Convert to appropriate integer data types
    master_table['Year'] = master_table['Year'].astype(np.int16)
    master_table['Month'] = master_table['Month'].astype(np.uint8)
    master_table['Day'] = master_table['Day'].astype(np.uint8)
    master_table['Hour'] = master_table['Hour'].astype(np.uint8)
    master_table['WeekOfMonth'] = master_table['WeekOfMonth'].astype(np.uint8)
    master_table['Quarter'] = master_table['Quarter'].astype(np.uint8)
    master_table['DayOfYear'] = master_table['DayOfYear'].astype(np.uint16)

    return master_table

In [19]:
master_table=create_master_table(energy_data, temperature_data)
master_table

,Region,TotalDemand,Year,Month,Day,Hour,Minute,Precipitation,RelativeHumidity%,AirTemperature,WetBulbTemperature,DewTemperature,SeaPressure,StationPressure,DayOfWeek,WeekOfMonth,Quarter,DayOfYear,Season,TimeOfDay,IsWeekend,TemperatureDifference,PressureDifference,DayNightTempDifference,TemperatureLevel,PrecipitationLevel,PressureLevel,HumidityLevel
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-01 00:30:00,NSW,6763.57000,2000,1,1,0,30.0,0.0,65.0,13.7,14.1,6.8,1011.1,1006.4,Saturday,1,1,1,Summer,Night,True,6.9,4.7,2.26832,Mild,No Precipitation,Normal Pressure,Moderate Moist
2000-01-01 00:30:00,QLD,3905.56833,2000,1,1,0,30.0,0.0,65.0,13.7,19.2,6.8,1007.2,1006.2,Saturday,1,1,1,Summer,Night,True,6.9,1.0,2.26832,Mild,No Precipitation,Normal Pressure,Moderate Moist
2000-01-01 00:30:00,SA,1328.68667,2000,1,1,0,30.0,0.0,65.0,13.2,10.1,6.8,1018.7,1012.6,Saturday,1,1,1,Summer,Night,True,6.4,6.1,2.26832,Mild,No Precipitation,Normal Pressure,Moderate Moist
2000-01-01 00:30:00,VIC,4419.03667,2000,1,1,0,30.0,0.0,65.0,13.7,10.5,7.2,1017.0,1013.1,Saturday,1,1,1,Summer,Night,True,6.5,3.9,2.26832,Mild,No Precipitation,Normal Pressure,Moderate Moist
2000-01-01 01:00:00,VIC,4312.54000,2000,1,1,1,0.0,0.8,63.0,13.6,10.3,6.7,1016.8,1012.9,Saturday,1,1,1,Summer,Night,True,6.9,3.9,2.26832,Mild,Light Precipitation,Normal Pressure,Moderate Moist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 00:00:00,TAS,1006.70000,2020,1,1,0,0.0,0.0,61.0,18.8,9.5,8.7,1014.9,1004.6,Wednesday,1,1,1,Summer,Night,False,10.1,10.3,2.26832,Mild,No Precipitation,Normal Pressure,Moderate Moist
2020-01-01 00:00:00,QLD,6218.39000,2020,1,1,0,0.0,0.0,61.0,18.8,21.6,8.7,1014.9,1013.8,Wednesday,1,1,1,Summer,Night,False,10.1,1.1,2.26832,Mild,No Precipitation,Normal Pressure,Moderate Moist
2020-01-01 00:00:00,SA,1474.11000,2020,1,1,0,0.0,0.0,61.0,18.8,12.3,8.7,1014.9,1010.6,Wednesday,1,1,1,Summer,Night,False,10.1,4.3,2.26832,Mild,No Precipitation,Normal Pressure,Moderate Moist


In [20]:
# Save master_table to a pickle file
master_table.to_pickle('master_table.pkl')

# 09- Encoding categorical variables

The foloowing function one_hot_encode_columns takes a DataFrame master_table as input and performs one-hot encoding on specified categorical columns: 'DayOfWeek', 'Season', 'TimeOfDay', 'IsWeekend', 'IsSummer', 'IsWinter', 'IsDaytime', 'IsNighttime', 'IsHoliday', 'TemperatureLevel', 'PrecipitationLevel', 'PressureLevel', and 'HumidityLevel'. It begins by creating a copy of the original DataFrame to preserve the original data. Then, it initializes a OneHotEncoder object from scikit-learn. The fit_transform method of the OneHotEncoder is applied to the specified categorical columns to generate the encoded columns. The encoded column names are retrieved using the get_feature_names_out method. A new DataFrame, encoded_df, is created using the encoded columns and their names. Next, the original categorical columns are dropped from the data_encoded DataFrame. The index of both dataframes is reset, and then they are concatenated along the columns axis. Finally, a new 'Date' column is created by combining the 'Year', 'Month', 'Day', and 'Hour' columns and set as the index of the resulting DataFrame before returning it.

In [21]:
master_table.columns

Index(['Region', 'TotalDemand', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Precipitation', 'RelativeHumidity%', 'AirTemperature',
       'WetBulbTemperature', 'DewTemperature', 'SeaPressure',
       'StationPressure', 'DayOfWeek', 'WeekOfMonth', 'Quarter', 'DayOfYear',
       'Season', 'TimeOfDay', 'IsWeekend', 'TemperatureDifference',
       'PressureDifference', 'DayNightTempDifference', 'TemperatureLevel',
       'PrecipitationLevel', 'PressureLevel', 'HumidityLevel'],
      dtype='object')

In [22]:
master_table.dtypes

Region                    category
TotalDemand                float64
Year                         int16
Month                        uint8
Day                          uint8
Hour                         uint8
Minute                     float64
Precipitation              float64
RelativeHumidity%          float64
AirTemperature             float64
WetBulbTemperature         float64
DewTemperature             float64
SeaPressure                float64
StationPressure            float64
DayOfWeek                   object
WeekOfMonth                  uint8
Quarter                      uint8
DayOfYear                   uint16
Season                      object
TimeOfDay                   object
IsWeekend                     bool
TemperatureDifference      float32
PressureDifference         float32
DayNightTempDifference     float32
TemperatureLevel          category
PrecipitationLevel        category
PressureLevel             category
HumidityLevel             category
dtype: object

In [23]:
def one_hot_encode_columns(master_table):
     
    # Copying the DataFrame
    data_encoded = master_table.copy()
    
    # Initializing OneHotEncoder
    onehot_encoder = OneHotEncoder()
    
    # The fit_transform method of the OneHotEncoder object is appling to the specified categorical columns
    encoded_cols = onehot_encoder.fit_transform(master_table[['DayOfWeek', 'Season','TimeOfDay', 'IsWeekend','TemperatureLevel','PrecipitationLevel', 'PressureLevel',
                                                              'HumidityLevel']])
    
    # Retrieving Encoded Column Names
    encoded_column_names = onehot_encoder.get_feature_names_out(['DayOfWeek', 'Season','TimeOfDay', 'IsWeekend','TemperatureLevel','PrecipitationLevel', 
                                                                 'PressureLevel','HumidityLevel'])
    # Creating Encoded DataFrame
    encoded_df = pd.DataFrame(encoded_cols.toarray(), columns=encoded_column_names)
    
    # Dropping Original Categorical Columns
    data_encoded = data_encoded.drop(['DayOfWeek', 'Season','TimeOfDay', 'IsWeekend', 'TemperatureLevel','PrecipitationLevel', 'PressureLevel',
                                      'HumidityLevel'], axis=1)
    
    # Resetting Index
    encoded_df.reset_index(drop=True, inplace=True)
    data_encoded.reset_index(drop=True, inplace=True)
    
    # Concatenating DataFrames
    data_encoded = pd.concat([data_encoded, encoded_df], axis=1)

    # Create the "Date" column from "Year", "Month", "Day", "Hour", and "Minute" columns and set as the index
    data_encoded["Date"] = pd.to_datetime(data_encoded[["Year", "Month", "Day", "Hour"]])
    data_encoded.set_index("Date", inplace=True)
    
    return data_encoded 

In [24]:
data_encoded= one_hot_encode_columns(master_table)

In [25]:
data_encoded.columns

Index(['Region', 'TotalDemand', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Precipitation', 'RelativeHumidity%', 'AirTemperature',
       'WetBulbTemperature', 'DewTemperature', 'SeaPressure',
       'StationPressure', 'WeekOfMonth', 'Quarter', 'DayOfYear',
       'TemperatureDifference', 'PressureDifference', 'DayNightTempDifference',
       'DayOfWeek_Friday', 'DayOfWeek_Monday', 'DayOfWeek_Saturday',
       'DayOfWeek_Sunday', 'DayOfWeek_Thursday', 'DayOfWeek_Tuesday',
       'DayOfWeek_Wednesday', 'Season_Autumn', 'Season_Spring',
       'Season_Summer', 'Season_Winter', 'TimeOfDay_Afternoon',
       'TimeOfDay_Evening', 'TimeOfDay_Morning', 'TimeOfDay_Night',
       'IsWeekend_False', 'IsWeekend_True', 'TemperatureLevel_Cold',
       'TemperatureLevel_Hot', 'TemperatureLevel_Mild',
       'TemperatureLevel_Very Cold', 'TemperatureLevel_Warm',
       'PrecipitationLevel_Heavy Precipitation',
       'PrecipitationLevel_Light Precipitation',
       'PrecipitationLevel_Moderate 

# 10- Resampling each group to an hourly frequency

The following function groups the data by the specified region column and resamples it to hourly intervals, calculating the mean value for each interval. This resampling process effectively aggregates the data to hourly resolution, which can be beneficial for various analyses, especially when aligning energy consumption and temperature data for further analysis or modeling. Furthermore, it effectively reduced the potential risk of overfitting caused by including excessively detailed temporal information, and it can help reduce the impact of anomalies or outliers present in the original data. After resampling, the function resets the index to include the region column and sorts the data by index.

In [26]:
def resample_data_to_hourly(data_encoded):
    
    # Group by region and resample to hourly intervals, calculating mean
    resampled_data = data_encoded.groupby("Region").resample("H").mean().reset_index("Region").sort_index()
    
     # Drop unnecessary columns
    resampled_data.drop(["Minute"], axis=1, inplace=True)
    
    # Drop rows with missing values
    resampled_data.dropna(inplace=True)
    
    # Convert to appropriate integer data types
    resampled_data['Year'] = resampled_data['Year'].astype(np.int16)
    resampled_data['Month'] = resampled_data['Month'].astype(np.uint8)
    resampled_data['Day'] = resampled_data['Day'].astype(np.uint8)
    resampled_data['Hour'] = resampled_data['Hour'].astype(np.uint8)
    resampled_data['WeekOfMonth'] = resampled_data['WeekOfMonth'].astype(np.uint8)
    resampled_data['Quarter'] = resampled_data['Quarter'].astype(np.uint8)
    resampled_data['DayOfYear'] = resampled_data['DayOfYear'].astype(np.uint16)
    
    # List of columns to convert to integer
    binary_columns = ['DayOfWeek_Friday', 'DayOfWeek_Monday', 'DayOfWeek_Saturday', 'DayOfWeek_Sunday','DayOfWeek_Thursday', 
                      'DayOfWeek_Tuesday', 'DayOfWeek_Wednesday', 'Season_Autumn','Season_Spring', 'Season_Summer', 
                      'Season_Winter', 'TimeOfDay_Afternoon', 'TimeOfDay_Evening','TimeOfDay_Morning', 'TimeOfDay_Night',
                      'IsWeekend_False', 'IsWeekend_True', 'TemperatureLevel_Cold', 'TemperatureLevel_Hot', 'TemperatureLevel_Mild',
                      'TemperatureLevel_Very Cold', 'TemperatureLevel_Warm','PrecipitationLevel_Heavy Precipitation', 'PrecipitationLevel_Light Precipitation',
                      'PrecipitationLevel_Moderate Precipitation', 'PrecipitationLevel_No Precipitation','PressureLevel_High Pressure', 
                      'PressureLevel_Low Pressure', 'PressureLevel_Normal Pressure','HumidityLevel_High Moist', 'HumidityLevel_Low Moist', 
                      'HumidityLevel_Moderate Moist']

    # Convert the binary columns to integer type
    resampled_data[binary_columns] = resampled_data[binary_columns].astype('int16')
    
    # Filter the data from 2010 onwards
    resampled_data = resampled_data.loc['2010':]

    return resampled_data

In [27]:
resampled_data = resample_data_to_hourly(data_encoded)

In [28]:
# Save data to a pickle file
resampled_data.to_pickle('resampled_data.pkl')

---